# Student

In [1]:
import numpy as np

In [246]:
def evaluate(skills, subjs):
    val = 0
    for subj in subjs:
        val += skills.get(subj, 0)
    return val / len(subjs)

class World:
    def __init__(self, subj_num, seed=None):
        np.random.seed(seed)
        self.subjs = ["%d+%d=%d" % (i+1,j+1,i+j+2) for i in range(subj_num) for j in range(subj_num)]
    
    def __repr__(self):
        return "%s(%r)" % (self.__class__, self.__dict__)
    
    def random_sim_subjs(self, group_num):
        def split(subjs, n):
            g = [[] for _ in range(n)]
            for s in subjs:
                i = np.random.randint(n)
                g[i].append(s)
            return g

        groups = split(self.subjs, group_num)
        sims = {}
        for group in groups:
            for subj in group:
                sims[subj] = [sim for sim in group if sim!=subj]
        return sims

    def evaluate(self, student, prn=False):
        if type(student) is list:
            v = 0
            for s in student:
                v += self.evaluate(s, prn=False)
            v /= len(student)
            if prn:
                print(v)                
            return v
        
        v = evaluate (student.skills, self.subjs)
        if prn:
            print(v)
        return v
    
    def dirichlet_sim_subjs(self, group_num, a):
        dist = np.random.dirichlet([a]*group_num)
        def split(subjs, n):
            g = [[] for _ in range(n)]
            for s in subjs:
                r = np.random.random()
                q = 0.
                for i,d in enumerate(dist):
                    q += d
                    if r < q:
                        break
                g[i].append(s)
            return g
        groups = split(self.subjs, group_num)
        sims = {}
        for group in groups:
            for subj in group:
                sims[subj] = [sim for sim in group if sim!=subj]
        return sims
        
    def kolhoz_sim_subjs(self):
        groups = [[] for _ in range(len(self.subjs)//2+1)]
        k = np.random.randint(len(groups))
        for s in self.subjs:
            if np.random.random()<1/2:
                i = np.random.randint(len(groups))
                groups[i].append(s)
            else:
                groups[k].append(s)
        
        sims = {}
        for group in groups:
            for subj in group:
                sims[subj] = [sim for sim in group if sim!=subj]
        return sims
                

In [247]:
world = World(subj_num=10, seed=None)
sims = world.kolhoz_sim_subjs()

for k,v in sims.items():
    pass
    #print(k, len(v)+1)

In [248]:
class BaseTeacher:
    def __init__(self, subjs):
        self.subjs = subjs

    def teach(self, student):
        if type(student) is list:
            for s in student:
                self.teach(s)
        else:
            self._do_teach(student)
    
    def _do_teach(self, student):
        raise NotImplementedError

        
class RandomTeacher(BaseTeacher):
    def _do_teach(self, student):
        subj = np.random.choice(self.subjs)
        student.learn(subj)

In [249]:
import uuid

class Student:
    def __init__(self, sim_subjs, learn_rate, forget_rate, sim_rate):
        self.id = str(uuid.uuid4())
        self.skills = {}
        self.sim_subjs = sim_subjs
        self.lr = learn_rate
        self.fr = forget_rate
        self.sr = sim_rate

    def clear(self):
        for s in self.skills.keys():
            self.skills[s] = 0

    def learn(self, subj):
        # update subj skill
        v = self.skills.get(subj, 0)
        self.skills[subj] = min(v+self.lr, 1)
        # update similar skills
        for sim in self.sim_subjs.get(subj, []):
            v = self.skills.get(sim, 0)
            self.skills[sim] = min(v+self.lr*self.sr, 1)

    def forget(self):
        for s,v in self.skills.items():
            self.skills[s] = v*(1-self.fr)
            
    def __repr__(self):
        return "%s(%r)" % (self.__class__, self.__dict__)
   

In [250]:
class Profile:
    def __init__(self, sim_subjs, learn_rate, forget_rate, sim_rate):
        self.sim_subjs   = sim_subjs
        self.learn_rate  = learn_rate
        self.forget_rate = forget_rate
        self.sim_rate    = sim_rate
        

In [251]:
from copy import copy, deepcopy

class NaiveTeacher(BaseTeacher):
    def __init__(self, subjs):
        super().__init__(subjs)
        self.students={}
                
    def _do_teach(self, student):
        best_subj = None
        best_val  = -1
        
        prev_skills = copy(student.skills)

        for subj in self.subjs:
            student.learn(subj)
            val = evaluate(student.skills, self.subjs)
            if val > best_val:
                best_subj = subj
                best_val  = val
            student.skills = copy(prev_skills)                
        student.learn(best_subj)

In [252]:
world = World(subj_num=3, seed=None)

profiles = [Profile(world.dirichlet_sim_subjs(10, a=0.999), learn_rate=.1, forget_rate=.0, sim_rate=1.)
          for _ in range(1)]

profiles = [Profile(world.kolhoz_sim_subjs(), learn_rate=.9999, forget_rate=.0, sim_rate=1.)
          for _ in range(2)]

print(profiles[0].sim_subjs)

students = [Student(p.sim_subjs, p.learn_rate, p.forget_rate, p.sim_rate) 
           for p in profiles for _ in range(1)]

teachers = [T(subjs=world.subjs) for T in [RandomTeacher, NaiveTeacher]]

for t in teachers:
    for s in students:
        s.clear()
    for s in students:
        for i in range(2):
            t.teach(s)
            s.forget()
    print("%-14s: %.1f%%" % (t.__class__.__name__, world.evaluate(students)*100))
    

{'1+1=2': ['1+2=3', '1+3=4', '2+1=3', '2+2=4', '2+3=5', '3+1=4', '3+2=5', '3+3=6'], '1+2=3': ['1+1=2', '1+3=4', '2+1=3', '2+2=4', '2+3=5', '3+1=4', '3+2=5', '3+3=6'], '1+3=4': ['1+1=2', '1+2=3', '2+1=3', '2+2=4', '2+3=5', '3+1=4', '3+2=5', '3+3=6'], '2+1=3': ['1+1=2', '1+2=3', '1+3=4', '2+2=4', '2+3=5', '3+1=4', '3+2=5', '3+3=6'], '2+2=4': ['1+1=2', '1+2=3', '1+3=4', '2+1=3', '2+3=5', '3+1=4', '3+2=5', '3+3=6'], '2+3=5': ['1+1=2', '1+2=3', '1+3=4', '2+1=3', '2+2=4', '3+1=4', '3+2=5', '3+3=6'], '3+1=4': ['1+1=2', '1+2=3', '1+3=4', '2+1=3', '2+2=4', '2+3=5', '3+2=5', '3+3=6'], '3+2=5': ['1+1=2', '1+2=3', '1+3=4', '2+1=3', '2+2=4', '2+3=5', '3+1=4', '3+3=6'], '3+3=6': ['1+1=2', '1+2=3', '1+3=4', '2+1=3', '2+2=4', '2+3=5', '3+1=4', '3+2=5']}
RandomTeacher : 66.7%
NaiveTeacher  : 77.8%
